In [ ]:
import numpy as np
import numpy.linalg as npl

import holoviews as hv
hv.extension('matplotlib')

In [ ]:
np.random.seed(0)

In [ ]:
from linop_FourierTransform import FourierTransform
from gradient_descent import gradient_descent

In [ ]:
def unit_circle_points(N, dist=1):
    roots = np.exp(1j * np.linspace(0, 2 * np.pi, num=N, endpoint=False))
    X = np.empty((N, 2))
    X[:, 0] = np.real(roots)
    X[:, 1] = np.imag(roots)
    return X * dist

In [ ]:
d = 2
a0 = np.array([1])
t0 = np.array([[0] * d])

In [ ]:
def plot_traj(t0, t_init, traj_t, lim=None, opts_spe={}):
    size = 300

    for idx in range(traj_t.shape[1]):
        curve_traj = hv.Curve(traj_t[:, idx, :], label='Trajectory').opts(linestyle='-', marker='o', alpha=.3, ms=size // 25)
        point_init = hv.Points(t_init, label='Initialization').opts(color='r', marker='P', s=size)
        if idx == 0:
            layout = (curve_traj * point_init)
        else:
            layout = layout * (curve_traj * point_init)

    layout = layout * hv.Points(t0, label='Ground truth').opts(color='g', marker='X', s=size)

    opts = {'show_legend': True, 'fig_size': 200, 'fontscale': 2}
    for k, v in opts_spe.items():
        opts[k] = v
    

    if lim is None:
        opts['padding'] = 0.1
    else:
        opts['xlim'] = (-0.25, 0.25)
        opts['ylim'] = (-0.25, 0.25)

    output = layout.opts(**opts)

    hv.output(output)

In [ ]:
def plot_error(errors, opts_spe={}):
    opts = {'xlabel': r'Gradient Descent iterations', 
            'ylabel': r'Norm of residue $ \| y - Ax \|_{2}$', 
            'logy': True, 'fig_size': 500,
            'fontscale': 2, 'xlim': (0, len(errors))}
    
    for k, v in opts_spe.items():
        opts[k] = v
    
    output = hv.Curve(errors).opts(**opts)
        
    hv.output(output)

In [ ]:
def plot_eigenvalue(eigen_min, condition, opts_spe={}):
    curve_eigen_min = hv.Curve(eigen_min).opts(xlabel='Gradient Descent iterations', ylabel='Smallest eigenvalue', 
                                               fontscale=2)
    curve_condition = hv.Curve(condition).opts(xlabel='Gradient Descent iterations', ylabel='Condition number', 
                                               fontscale=2)

    opts = {'shared_axes': False, 'fig_size': 200,}
    for k, v in opts_spe.items():
        opts[k] = v
        
    output = (curve_eigen_min + curve_condition).opts(**opts)
    
    hv.output(output)

In [ ]:
def formatter_eig(value):
    return f"{-value:#.0e}"

def formatter_cond(value):
    return f"{value:#.0e}"
    
def plot_eigenvalue_log(eigen_min, condition, opts_spe={}):
    fontscale=1.5
    curve_eigen_min = hv.Curve(eigen_min).opts(xlabel='Gradient Descent iterations', ylabel='Smallest eigenvalue', 
                                               fontscale=fontscale, logy=True, invert_yaxis=True,
                                               yformatter=formatter_eig, show_title=False)
    curve_condition = hv.Curve(condition).opts(xlabel='Gradient Descent iterations', ylabel='Condition number', 
                                               fontscale=fontscale, logy=True, 
                                               yformatter=formatter_cond, show_title=False)

    opts = {'shared_axes': False, 'fig_size': 200,}
    for k, v in opts_spe.items():
        opts[k] = v
    output = (curve_eigen_min + curve_condition).opts(**opts)

    hv.output(output)

# Eigen values with $ x = a \delta_{t} $

In [ ]:
k = 1
m = k * 1000

a = np.ones(k) / k
t = unit_circle_points(k, dist=.2)

linop = FourierTransform(m=m, d=d, lamb=.2)
y = linop.Ax(a0, t0)

In [ ]:
a_est, t_est, traj_a, traj_t, errors = gradient_descent(y, linop, a, t, project=False, nit=500, tau={'min': -10, 'max': 10}, clip=False)

In [ ]:
plot_traj(t0, t, traj_t[:50], lim=True) 

In [ ]:
max_it = 500
plot_error(errors[:max_it])

In [ ]:
eigen_min = {}
condition = {}
for it in range(0, max_it, 1):
    H = linop.Hessian(traj_a[it], traj_t[it], y)
    condition[it] = npl.cond(H)
    eigen_min[it] = np.min(npl.eigvals(H))

In [ ]:
plot_eigenvalue(eigen_min, condition)

# Eigen values with $ x = a_1 \delta_{t_{1}} + a_2 \delta_{t_{2}} $

In [ ]:
k, d = 2, 2
m = k * 40
a0 = np.array([1])
t0 = np.array([[0] * d])

a = np.ones(k) / k
t = unit_circle_points(k, dist=.2)

linop = FourierTransform(m=m, d=d, lamb=.2)
y = linop.Ax(a0, t0)

In [ ]:
a_est, t_est, traj_a, traj_t, errors = gradient_descent(y, linop, a, t, project=False, clip=False,
                                                        tau={'min': -10, 'max': 10}, nit=10_000)

In [ ]:
plot_traj(t0, t, traj_t, lim=True)

In [ ]:
eigen_min = {}
condition = {}
for it in range(0, 10001, 20):
    H = linop.Hessian(traj_a[it], traj_t[it], y)
    condition[it] = npl.cond(H)
    eigen_min[it] = -np.min(npl.eigvals(H))

In [ ]:
plot_eigenvalue_log(eigen_min, condition)

## Eigen values with $ x = \sum_{i = 1}^{5} a_i \delta_{t_{i}} $

In [ ]:
k, d = 5, 2
m = k * 40
a0 = np.array([1])
t0 = np.array([[0] * d])

a = np.ones(k) / k
t = unit_circle_points(k, dist=.2)

linop = FourierTransform(m=m, d=d, lamb=.2)
y = linop.Ax(a0, t0)

In [ ]:
a_est, t_est, traj_a, traj_t, errors = gradient_descent(y, linop, a, t, project=False, clip=False,
                                                        tau={'min': -10, 'max': 10}, nit=10_000)

In [ ]:
plot_traj(t0, t, traj_t, lim=True)

In [ ]:
plot_error(errors)

In [ ]:
eigen_min = {}
condition = {}
for it in range(0, 10001, 20):
    H = linop.Hessian(traj_a[it], traj_t[it], y)
    condition[it] = npl.cond(H)
    eigen_min[it] = -np.min(npl.eigvals(H))

In [ ]:
plot_eigenvalue_log(eigen_min, condition)

# Eigen values with $ x = \sum_{i = 1}^{5} a_i \delta_{t_{i}} $

In [ ]:
k, d = 5, 2
m = k * 40
a0 = np.array([1])
t0 = np.array([[0] * d])

a = np.ones(k) / k
t = unit_circle_points(k, dist=1e-5)

linop = FourierTransform(m=m, d=d, lamb=.2)
y = linop.Ax(a0, t0)

In [ ]:
a_est, t_est, traj_a, traj_t, errors = gradient_descent(y, linop, a, t, project=False, clip=False,
                                                        tau={'min': -15, 'max': 15}, nit=1000)

In [ ]:
def formatter(value):
    if value == 0:
        return 0
    elif value > 0:
        return f"{str(value)[-1]}.e-6"
    else:
        return f"-{str(value)[-1]}.e-6"

def plot_traj(t0, t_init, traj_t, lim=None, save=None, opts_spe={}):
    size = 300
    xticks = [(5e-6, '5.e-6'), (0, 0), (-5e-6, '-5.e-6')]
    yticks = [(6e-6, '6.e-6'), (2e-6, '2.e-6'), (0, 0), (-2e-6, '-2.e-6'), (-6e-6, '-6.e-6')]

    for idx in range(traj_t.shape[1]):
        curve_traj = hv.Curve(traj_t[:, idx, :], label='Trajectory').opts(linestyle='-', marker='o', alpha=.3, ms=size // 25, 
                                                                          xticks=xticks, yticks=yticks)
        point_init = hv.Points(t_init, label='Initialization').opts(color='r', marker='P', s=size)
        if idx == 0:
            layout = (curve_traj * point_init)
        else:
            layout = layout * (curve_traj * point_init)

    layout = layout * hv.Points(t0, label='Ground truth').opts(color='g', marker='X', s=size)

    opts = {'show_legend': True, 'fig_size': 200, 'fontscale': 2, 'yformatter': formatter}
    for k, v in opts_spe.items():
        opts[k] = v
    

    if lim is None:
        opts['padding'] = 0.1
    else:
        opts['xlim'] = (-0.25, 0.25)
        opts['ylim'] = (-0.25, 0.25)

    output = layout.opts(**opts)
    if save:
        save_plot(output, save)

    hv.output(output)

In [ ]:
traj_t.shape

In [ ]:
plot_traj(t0, traj_t[2, :, :], traj_t[2:, :, :], lim=None, opts_spe=dict(legend_position='top_left'))

In [ ]:
plot_error(errors[2:])

In [ ]:
eigen_min = {}
condition = {}
for it in range(0, 1000, 1):
    H = linop.Hessian(traj_a[it], traj_t[it], y)
    condition[it] = npl.cond(H)
    eigen_min[it] = abs(np.min(np.real(npl.eigvalsh(H))))

In [ ]:
plot_eigenvalue_log(eigen_min, condition)